# Joe Stanley
### ECE 524 - HWK 3

In [1]:
# Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
import eepower as eep
from eepower import u, m, k, M

In [2]:
# Define Givens

# Per-Unit Bases
Sbase = 30*M
Vb1 = 138*k
Vb2 = 13.8*k
Vb3 = 4.16*k

# Source
S_src = 3500*M #VA
XR_src = 15 #unitless
V_src = 138*k #V
Z_src = (0.0005701629473466338+0.008552444210199505j) # per-unit (from prev. assignment)

# Transformer(s)
S_xfm = 30*M #VA
Z_xfm = 7.5/100 #pu
XR_xfm = 18.6 #unitless
V_busA = 13.8*k #V
Z_xfm = (0.00402644303154544+0.07489184038674519j) # per-unit (from prev. assignment)

# Generator
S_gen = 35.3*M #VA
Xdv_gen = 10/100 #pu
XR_gen = 48 #unitless
Z_gen = (0.0017705382436260624+0.08498583569405099j) # per-unit (from prev. assignment)

# Motor Load (each motor)
P_mot = 30000 #hp
Xd_mot = 16.7/100 #pu
XR_mot = 10 #unitless
pf_mot = 0.8 #lagging

# Cap Bank
S_cap = 10*M #VAr

# Load Transformer
S_xfm_ld = 5*M #VA
Z_xfm_ld = 5.5/100 #pu
XR_xfm_ld = 7 #unitless
V_ld = 4.16*k #V

In [22]:
C = eep.farads(6*M,13.8*k/np.sqrt(3))
print(C)
I,F = eep.capbacktoback(C1=C,C2=C,Lm=16*u,VLL=13.8*k)

0.00025071667153732727
